## Pipeline обработки запроса

### Библиотеки

In [ ]:
from src.utils import setup_environment, load_data
from src.config import PipelineConfig
from src.engine import PipelineEngine
from src.state import PipelineStatus
from openai import OpenAI

### Загрузка данных и API

In [ ]:
api_key, db_path = setup_environment()

client = OpenAI(
    base_url="https://openrouter.ai/api/v1", 
    api_key=api_key
)

df = load_data(db_path, wave_filter=["2025-03"])
print(f"Dataset loaded. Shape: {df.shape}")

### Конфигурация моделей

In [ ]:
# Настраиваем конфиг
PPL_cfg = PipelineConfig.setup(
    df=df,
    question_waves=["2025-03"],
    client=client,
    # Настройки агентов
    retriever_params={
        "model": "x-ai/grok-4.1-fast",
        "temperature": 0.5, 
        "provider_sort": "price"
    },
    planner_params={
        "model": "x-ai/grok-4.1-fast",
        "temperature": 0.2,
        "max_tokens": 16_000
    }
)

# Создаем движок
engine = PipelineEngine(config=PPL_cfg, base_log_dir="logs")

### Запрос пользователя

In [ ]:
# user_query = "Мне нужно рассчитать средний доход людей старше 30 из Москвы"
# user_query = "Рассчитай среднюю зарплату женщин с высшим образованием"
# user_query = "Посчитай средний доход семьи среди респондентов из Центрального ФО"
# user_query = "Мне нужно узнать средний возраст работающих респондентов"
# user_query = "Рассчитай количество респондентов, делающих покупки в Чижике"
# user_query = "Сколько человек совершают покупки в Магните чаще 3 раз в неделю"
# user_query = "Мне нужно узнать средний чек покупателей в ВкусВилл"
# user_query = "Рассчитай количество респондентов, использующих промоакции в Ленте"
# user_query = "Я хочу посчитать размер средних сбережений и норму сбережений среди тех, у кого они есть"
# user_query = "Рассчитай среднюю долю трат на продукты питания по доходным группам"
# user_query = "Посчитай средние траты на здоровье среди семей с детьми"
# user_query = "Мне нужен глубокий анализ связи туризма и доходов людей"
# user_query = "Проанализируй связь между уровнем образования и размером сбережений"
# user_query = "Изучи зависимость между доходом и выбором продуктовых магазинов"
# user_query = "Мне нужен анализ связи между семейным положением и потребительским поведением"
# user_query = "Проанализируй, как возраст влияет на использование онлайн-доставки продуктов"
# user_query = "Как обстоит ситуация с занятостью в различных федеральных округах"
# user_query = "Проанализируй портрет покупателя дискаунтеров по сравнению с премиальными сетями"
# user_query = "Проанализируй изменение потребительского поведения среди тех, кто активно экономит на продуктах"
# user_query = "Изучи различия в потребительских предпочтениях между Москвой и регионами"
# user_query = "Проанализируй изменение потребительского поведения среди тех, кто активно экономит на продуктах"
user_query = "Сравни популярность брендов среди мужчин старше 40 лет"

Создаем новую сессию

In [ ]:
session = engine.create_session(user_query)

print(f"🚀 Session ID: {session.id}")
print(f"📂 Work Dir:  {session._work_dir}")

Или подгружаем имеющуюся

In [ ]:
session_path = "logs/run_2025-12-12_17-05-04_ce9ecfaf" 
session = engine.load_session(session_path)

print(f"Loaded session: {session.id}")
print(f"Current status: {session.status}")
print(f"Query: {session.user_query}")

Если статус был FAILED, можно сделать rewind и продолжить

In [ ]:
# # Статус до которого нужно откатить
# rewind_to = PipelineStatus.GROUNDED

# if session.status == PipelineStatus.FAILED:
#     session = engine.rewind(session, rewind_to)
#     session = engine.step(session)

### Retriever

In [ ]:
# # Откат к предыдущему состоянию
# session = engine.rewind(session, target_status=PipelineStatus.CREATED)

# # Замена параметров моделей
# engine.cfg.retriever_config.model = "..."
# engine.cfg.retriever_config.temperature = 0.7

session = engine.step(session)

print(f"Status: {session.status}")
print("-" * 30)

if session.retriever_output:
    print(session.retriever_output)
else:
    print("Ничего не найдено :(")

### Planner

In [ ]:
# Откат к предыдущему состоянию
session = engine.rewind(session, target_status=PipelineStatus.RETRIEVED)

# Замена параметров моделей
# engine.cfg.planner_config.model = "..."
engine.cfg.planner_config.temperature = 0.2
engine.cfg.planner_config.max_tokens = 16_000

session = engine.step(session)

print(f"Status: {session.status}")
print("-" * 30)

if session.planner_output:
    print(session.planner_output)

### Grounder

In [ ]:
# # Откат к предыдущему состоянию
# session = engine.rewind(session, target_status=PipelineStatus.PLANNED)

session = engine.step(session)

print(f"Status: {session.status}")

### Executor

In [ ]:
# # Откат к предыдущему состоянию
# session = engine.rewind(session, target_status=PipelineStatus.GROUNDED)

session = engine.step(session)

print(f"Status: {session.status}")

if session.status == PipelineStatus.EXECUTED:
    print("✅ Готово!")
    print(f"📊 Результат: {session.execution_result_path}")
elif session.status == PipelineStatus.FAILED:
    print("❌ Ошибка выполнения. Проверьте log.txt в папке сессии.")